In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('signals_dataset').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
spark

In [4]:
par_df = spark.read.parquet("path_to_input_file")

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Lenovo5Pro\AppData\Local\Temp\ipykernel_4900\522719453.py:1: SyntaxWarning: invalid escape sequence '\T'
  par_df = spark.read.parquet("C:\Temp\signals\part-00000-8b43448f-2282-4fe6-a5cc-64cdf61b750e-c000.snappy.parquet")


In [5]:
par_df.show()

+---------+-------+------+--------+------------+
|entity_id|item_id|source|month_id|signal_count|
+---------+-------+------+--------+------------+
|   359781|  20001| 21000|  201705|           1|
|   359781|  20001| 21000|  201707|           1|
|   359781|      3| 21000|  201708|           2|
|   359781|      3| 21000|  201712|           2|
|   359781|      3|   975|  201512|           4|
|   359781|      3| 21000|  201710|           2|
|   359781|      3| 21000|  201705|           3|
|   359781|      3|   600|  201410|           1|
|   359781|      3| 21000|  201707|           3|
|   359781|  10002| 21000|  201705|           1|
|   359781|  10002| 21000|  201707|           1|
|   359781|  10001| 21000|  201705|           1|
|   359781|  10001| 21000|  201707|           1|
|135043968|  20000| 21000|  201706|           1|
|135043968|  20000| 21000|  201707|           1|
|135043968|      1| 21000|  201706|           1|
|135043968|      1| 10002|  201704|           1|
|135043968|      1| 

In [6]:
from pyspark.sql.functions import col, min, max, sum as spark_sum, collect_list, desc

# Aggregate to find oldest and newest month_id for each entity_id and item_id
agg_df = par_df.groupBy("entity_id", "item_id").agg(min("month_id").alias("oldest_month_id"),
                        max("month_id").alias("newest_month_id"),
                        spark_sum("signal_count").alias("total_signals"))

# Find the oldest and newest month_id
oldest_item_df = agg_df.orderBy("oldest_month_id", "item_id").groupBy("entity_id").agg(collect_list("item_id")[0].alias("oldest_item_id"))
newest_item_df = agg_df.orderBy(desc("newest_month_id"), "item_id").groupBy("entity_id").agg(collect_list("item_id")[0].alias("newest_item_id"))

# Join the oldest and newest 
result_df = oldest_item_df.join(newest_item_df, "entity_id", "inner")

# Join signals sums to the final result df
result_df = result_df.join(agg_df.groupBy("entity_id").agg(spark_sum("total_signals").alias("total_signals")), "entity_id", "inner")

result_df.show()





+---------+--------------+--------------+-------------+
|entity_id|oldest_item_id|newest_item_id|total_signals|
+---------+--------------+--------------+-------------+
|101554284|         10001|         10001|            2|
|130290759|             1|             1|          603|
|202066407|         20000|         20000|            1|
|211874530|             0|             0|           53|
|197565900|             2|             2|           54|
|171040125|         10001|         10001|            2|
|257416567|         10001|         10001|            2|
|190979372|             2|             1|           20|
|   680989|             0|             0|           32|
|179037429|         10001|         10001|            2|
|294275250|             3|             3|           11|
|189341470|         10001|         10001|            2|
| 65059978|             1|             1|          472|
|220897278|             2|             1|           66|
|218643663|         10001|         10001|       

In [24]:
result_df.write.parquet(r"path_to_output_folder", mode="overwrite")